#### This exercise was performed on ARCC Beartooth with the following environment/software packages:

#### Load Preinstalled Modules:
module load gcc12.2.0 miniconda3 git/2.37.0

#### Create environment then activate with:
conda env create -f whatev_ml_env.yml
conda activate whatev_ml_env

#### Make sure jupyter can see the kernel in your conda env so you can select from southpass interface dropdown:
python -m ipykernel install --user --name=whatev_conda

#### Download data to folder and extract
wget https://archive.ics.uci.edu/static/public/186/wine+quality.zip
unzip wine+quality.zip

wget https://www.openml.org/data/download/53537/primary-tumor.arff 

### In prior exercises, we performed exploratory data analysis and cleaned our data initially - now we will perform machine learning on the data which we've already cleaned.

## Directions
Choose a small number of different machine learning algorithms. For example, you
could use a random forest, support vector machine, linear/logistic regression, a
decision/regression tree learner, and gradient boosting. You will also have to
choose their hyperparameters, e.g. the default values. Determine the best
machine learning algorithm for your dataset, where the "best" algorithm could be
a set of algorithms. Make sure that the way you evaluate this avoids bias and
overfitting. You could use statistical tests to make this determination.